In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import bs4
import pandas as pd

chrome_options = Options()

chrome_options.add_argument("user-data-dir=C:\\Users\\exaylo\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")
driver = webdriver.Chrome(options=chrome_options)


driver.get("https://www.esportsearnings.com/players")   
time.sleep(10)
page = driver.page_source
soup = bs4.BeautifulSoup(page, 'html.parser')
lista = soup.find('tbody').find_all('tr')
data_list=  []
for gracz in lista:
    lp = gracz.find('td', class_='detail_list_order').text.replace('.','')
    #kraj = gracz.find('a' , href=lambda href: href and '/countries/' in href ).find('img').get('alt')
    nick = gracz.find('a' , href=lambda href: href and '/players/' in href ).text
    zarobki = gracz.find('td', class_='format_cell detail_list_prize border_right').text.strip().replace(',','')
    gra = gracz.find('td', class_='format_cell detail_list_game border_left').text
    data_list.append([lp, nick, zarobki, gra])

df = pd.DataFrame(data_list, columns=['lp', 'nick', 'zarobki', 'gra'])
df.set_index('lp', inplace=True)
df.to_csv('esportsearnings.csv')
driver.get("https://www.esportsearnings.com/games")   
page = driver.page_source
soup = bs4.BeautifulSoup(page, 'html.parser')
gry = soup.find_all('div', class_='detail_box_game')
data_list=  []
for gra in gry:
    lp = gra.find('div', class_='games_top_games_number').text.replace('#','')
    nazwa = gra.find('div', class_='games_top_games_title').text
    dane = gra.find('span', class_='front_page_highest_games_stats')
    data = dane.decode_contents().split('<br/>')
    pula_nagrod = data[0].strip() 
    liczba_graczy = data[1].split()[0].strip()  
    liczba_turniejow = data[2].split()[0].strip() 
    data_list.append([lp, nazwa, pula_nagrod, liczba_graczy, liczba_turniejow])
df = pd.DataFrame(data_list, columns=['lp', 'nazwa', 'pula_nagrod', 'liczba_graczy', 'liczba_turniejow'])
df.set_index('lp', inplace=True)
df.to_csv('gry.csv')
driver.quit()